In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from members import *

['Авраменко', 'Бубнова', 'Виноградова', 'Герасимов', 'Дятлов', 'Еналеев', 'Лебедев', 'Малышев', 'Ткачева']
Количество участников: 9
Количество женщин: 3


In [3]:
import numpy as np
import pandas as pd
import datetime

from collections import defaultdict
from collections import Counter

# Задача перечислить и распределить ресурсы

In [4]:
NBreakfast = 7
NDinner = 6
NLunch = 6
NSnack = 6

WomanFoodMaxWeight = 2000

In [15]:
Food = [# Продукты "ежедневного" потребления
         ('Соль'                      ,    7,    7,       Герасимов),
         ('Сахар'                     ,   60,    7,       ([Еналеев, Дятлов], [3.5, 3.5])),
         ('Чай (зеленый / черный)'    ,    7,    7,       Ткачева),
         ('Лимоны в сахаре'           ,   20,    7,       Дятлов),
         ('Перец'                     ,    1,    6,       Дятлов),
         ('Хмели-сунели'              ,    1,    6,       Дятлов),
         ('Аджика'                    ,    3,    6,       Ткачева),

        # Завтрак
        
         ('Геркулес Традиционный (завтрак)' ,   40,    2,       Герасимов),
         ('Рис (завтрак)'             ,   50,    2,       Герасимов),
         ('Гречка (завтрак)'          ,   50,    2,       Герасимов),
         ('Пшено (завтрак)'           ,   50,    1,       Герасимов),
         ('Сухое молоко (завтрак)'    ,   25,    7,       Авраменко),
         ('Колбаса (завтрак)'         ,   40,    2,       Герасимов),
         ('Грудинка (завтрак)'        ,   40,    3,       Герасимов),
         ('Сыр (завтрак)'             ,   40,    2,       Герасимов), 
         ('Изюм (завтрак)'            ,   5,     7,       Малышев),
         ('Чернослив (завтрак)'       ,   20,    1,       Ткачева),
         ('Cухое молоко в кофе (завтрак)'   ,   2,     7,       Авраменко),
         ('Кофе'                      ,   2,     7,       Ткачева),
     
        # Перекус
     
         ('Колбаса (перекус)'         ,   40,    3,       Авраменко),
         ('Сыр (перекус)'             ,   40,    3,       Авраменко),
         ('Шоколад (перекус)'         ,   25,    6,       Виноградова),
         ('Орехи (перекус)'           ,   20,    6,       Авраменко),
         ('Сухари (перекус)'          ,   40,    6,       ([Виноградова, Малышев, Авраменко], [2, 2, 2])), 
         ('Курага (перекус)'          ,   10,    6,       Ткачева),
         ('Чернослив (перекус)'       ,   10,    6,       Ткачева),
     
         # Обед
         
         ('Чеснок'                    ,    4,    6,       Герасимов),
         ('Супы'                      ,    35,   6,       Лебедев),
     
         # Ужин
     
         ('Рис (ужин)'                ,   80,    1,       Малышев),
         ('Гречка (ужин)'             ,   80,    2,       Малышев),
         ('Чечевица (ужин)'           ,   80,    1,       Малышев),
         ('Спагетти (ужин)'           ,   90,    2,       Малышев),
         ('Пеммикан (ужин)'           ,   81,    6,       Лебедев),
         ('Лук'                       ,   10,    6,       Малышев),
         ('Сладости (ужин)'           ,   40,    6,       Бубнова),
         ('Спирт (ужин)'              ,   15,    6,       Еналеев),
        
        # Бензин
        
         ('Бензин'                    ,   100,   7,       ([Еналеев, Дятлов], [3.9, 3.1]))
]


names_counter = Counter()
total_weight = 0.0

for m in Members.values():
    m.food = []

tbl = []    

for row in Food:
    one_portion_weight = float(row[1])
    portion_weight = one_portion_weight * NMembers
    times = float(row[2])
    weight = portion_weight * times
        
    total_weight += weight

    assert len(row) == 3 or len(row) == 4
    names_counter[row[0]] += 1
    assert names_counter[row[0]] <= 1

    
    
    if len(row) == 3:
        tbl.append({'name': row[0], 
                'portion': portion_weight,
                'times': times,
                'weight': weight})

    else:
        if isinstance(row[3], Member):
            member = row[3]
            assert member.name in MembersList
            member.food.append({'kind' : 'food', 'packing_weight': portion_weight, 
                                'weight' : weight, 'name' : row[0]})
            
            tbl.append({'name': row[0], 
                'portion': portion_weight,
                'times': times,
                'weight': weight, 'Кто несет': member.name})
        else:
            members, ks = row[3]
            assert np.sum(ks) - times < 1e-9, print(np.sum(ks), times)
            assert len(members) == len(ks)
            assert len(members) > 1
            
            for member, k in zip(members, ks):
                assert member.name in MembersList
                member.food.append({'kind' : 'food', 'packing_weight': portion_weight, 
                                    'weight' : portion_weight * k, 'name' : row[0]})
                
                tbl.append({'name': row[0], 
                            'portion': portion_weight,
                            'times': k,
                            'weight': portion_weight * k, 'Кто несет': member.name})
            
            if np.sum(ks) < times - 1e-9:
                # Есть нераспределенное снаряжение
                tbl.append({'name': row[0], 
                            'portion': portion_weight,
                            'times': times - np.sum(ks),
                            'weight': portion_weight * (times - np.sum(ks))})

            
print("Общий вес: {}".format(total_weight))

avg = (total_weight - WomanFoodMaxWeight * NWomans) / (NMembers - NWomans)
print("Средний вес на мужчину: {}".format(avg))
print("Средний вес на женщину: {}".format(WomanFoodMaxWeight))
print("Вес на человека в день: {}\n".format(total_weight / NMembers / ((NBreakfast + NDinner + NSnack + NLunch) / 4)))

cur_weight = 0.0
for name, member in sorted(Members.items()):
    w = np.sum([r['weight'] for r in member.food])
    cur_weight += w
    if name in Womans:
        d = WomanFoodMaxWeight - w
    else:
        d = avg - w
    
    print("{}: {}".format(name, d))
    
print("\nОстаток: {}".format(total_weight - cur_weight))

table = pd.DataFrame(data=tbl)
table

Общий вес: 42624.0
Средний вес на мужчину: 6104.0
Средний вес на женщину: 2000
Вес на человека в день: 757.76

Авраменко: 443.0
Бубнова: -160.0
Виноградова: -70.0
Герасимов: -43.0
Дятлов: 56.0
Еналеев: -106.0
Лебедев: -160.0
Малышев: 29.0
Ткачева: 11.0

Остаток: 0.0


,name,portion,times,weight,Кто несет
0,Соль,63.0,7.0,441.0,Герасимов
1,Сахар,540.0,3.5,1890.0,Еналеев
2,Сахар,540.0,3.5,1890.0,Дятлов
3,Чай (зеленый / черный),63.0,7.0,441.0,Ткачева
4,Лимоны в сахаре,180.0,7.0,1260.0,Дятлов
5,Перец,9.0,6.0,54.0,Дятлов
6,Хмели-сунели,9.0,6.0,54.0,Дятлов
7,Аджика,27.0,6.0,162.0,Ткачева
8,Геркулес Традиционный (завтрак),360.0,2.0,720.0,Герасимов
9,Рис (завтрак),450.0,2.0,900.0,Герасимов


### В 1.5л бутылке умещаятся 1150г бензина

In [17]:
for a, b in table.groupby(['Кто несет']):
        print("Участник: {}, несет: {}".format(a, b['weight'].sum()))
        print(str(b))
        print("\n\n")

with open("food.txt", 'w') as f:
    f.write("Дата создания: {}\n\n".format(datetime.datetime.now()))
    for a, b in table.groupby(['Кто несет']):
        f.write("Участник: {}, несет: {}\n".format(a, b['weight'].sum()))
        f.write(str(b))
        f.write("\n\n")
        
    f.write("\nСводная таблица\n")
    f.write(str(table))

Участник: Авраменко, несет: 5661.0
                             name  portion  times  weight  Кто несет
12         Сухое молоко (завтрак)    225.0    7.0  1575.0  Авраменко
18  Cухое молоко в кофе (завтрак)     18.0    7.0   126.0  Авраменко
20              Колбаса (перекус)    360.0    3.0  1080.0  Авраменко
21                  Сыр (перекус)    360.0    3.0  1080.0  Авраменко
23                Орехи (перекус)    180.0    6.0  1080.0  Авраменко
26               Сухари (перекус)    360.0    2.0   720.0  Авраменко



Участник: Бубнова, несет: 2160.0
               name  portion  times  weight Кто несет
37  Сладости (ужин)    360.0    6.0  2160.0   Бубнова



Участник: Виноградова, несет: 2070.0
                 name  portion  times  weight    Кто несет
22  Шоколад (перекус)    225.0    6.0  1350.0  Виноградова
24   Сухари (перекус)    360.0    2.0   720.0  Виноградова



Участник: Герасимов, несет: 6147.0
                               name  portion  times  weight  Кто несет
0           